In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import scipy.io as sio
from scipy import signal
import copy
from scipy.interpolate import griddata


---
# Detect muscle movements from EMG recordings
---


In [ ]:
# load data
matdat = sio.loadmat('EMGRT.mat')
emg = matdat['emg']
rts = np.squeeze(matdat['rts'])
timevec = np.squeeze(matdat['timevec'])

N = len(rts)

# example trials
trials2plot = [ 4, 23 ]


### a few plots to get an idea of the data

# RTs (button presses)
plt.subplot(121)
plt.plot(rts,'s-')
plt.xlabel('Trials')
plt.ylabel('Reaction times (ms)')

# histogram of RTs
plt.subplot(122)
plt.hist(rts,40)
plt.xlabel('Reaction times (ms)')
plt.ylabel('Count')
plt.show()


# EMG time series, sorted by RTs
sidx = np.argsort(rts)[::-1]

for i in range(N):
    plt.plot(timevec,emg[sidx[i],:]+i*1500,'k',linewidth=.2)

plt.xlabel('Time (ms)')
plt.ylabel('Trials')
plt.yticks([], [])
plt.show()



# plt two example trials
for i in range(0,2):
    
    # plot EMG trace
    plt.plot(timevec,emg[trials2plot[i],:],'r')
    
    # overlay button press time
    plt.plot([rts[trials2plot[i]], rts[trials2plot[i]]],[-6000, 6000],'k--')
    plt.title('Trial %d' %trials2plot[i])
    plt.show()


In [ ]:
# detect EMG onsets

# define baseline time window for normalization
baseidx = [0,0]
baseidx[0] = np.argmin((timevec--500)**2)
baseidx[1] = np.argmin((timevec)**2)

# pick z-threshold
zthresh = 100

# initialize outputs
emgonsets = np.zeros(N)

for triali in range(0,N):
    
    # convert to energy via TKEO
    tkeo = emg[triali,1:-1]**2 - emg[triali,0:-2]*emg[triali,2:]
    # for convenience, make the TKEO the same length as the time vector
    tkeo = np.append(0,tkeo)
    tkeo = np.append(tkeo,0)
    
    # convert to zscore from pre-0 activity
    tkeo = ( tkeo-np.mean(tkeo[range(baseidx[0],baseidx[1])]) ) / np.std(tkeo[range(baseidx[0],baseidx[1])])
    
    # find first suprathreshold point
    tkeoThresh = tkeo>zthresh
    tkeoThresh[timevec<0] = 0
    tkeoPnts = np.squeeze( np.where(tkeoThresh) )
    
    # grab the first suprathreshold point
    emgonsets[triali] = timevec[ tkeoPnts[0]+1 ]


In [ ]:
## more plots

# back to the EMG traces...
for i in range(2):
    
    # plot EMG trace
    plt.plot(timevec,emg[trials2plot[i],:],'r',label='EMG')
    
    # overlay button press time
    plt.plot([rts[trials2plot[i]], rts[trials2plot[i]]],[-6000, 6000],'k--',label='Button press')
    plt.title('Trial %d' %trials2plot[i])
    
    # overlay EMG onset
    xpnt = emgonsets[trials2plot[i]]
    plt.plot([xpnt,xpnt],[-6000, 6000],'b--',label='EMG onset')
    
    plt.legend()
    plt.show()
    